In [1]:
from nltk.corpus import reuters
import openai
from tqdm import tqdm
import numpy as np
import nltk
from openai.embeddings_utils import get_embeddings
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pickle
import librosa

with open('/users/jasper/oai.txt', 'r') as f:
    openai.api_key = f.read()

# nltk.download('reuters')

In [2]:
# new version
trade_docs = reuters.fileids(categories='trade')
crude_docs = reuters.fileids(categories='crude')

all_docs = [reuters.raw(doc_id) for doc_id in trade_docs + crude_docs]
all_labels = ['trade' for _ in trade_docs] + ['crude' for _ in crude_docs]

# shuffle docs and labels together
np.random.seed(42)
combined = list(zip(all_docs, all_labels))
np.random.shuffle(combined)
all_docs, all_labels = zip(*combined)

In [9]:
# get embeddings for train/test docs
print('Getting embeddings...')
embeddings_engine = "text-embedding-ada-002"
all_embeddings = get_embeddings(all_docs, engine=embeddings_engine)

# pickle embeddings
with open('embeddings/all_embeddings.pkl', 'wb') as f:
    pickle.dump(all_embeddings, f)

Getting embeddings...


In [10]:
# load embeddings
with open('embeddings/all_embeddings.pkl', 'rb') as f:
    all_embeddings = pickle.load(f)


In [11]:
# vanilla classification

# train/test split
X_train, X_test, y_train, y_test = train_test_split(all_embeddings, all_labels, test_size=0.2, random_state=42)

# train classifier
clf = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)
clf.fit(X_train, y_train)

# predict on test set
y_pred = clf.predict(X_test)

# evaluate
print(accuracy_score(y_test, y_pred))

Iteration 1, loss = 0.70101367
Iteration 2, loss = 0.68367065
Iteration 3, loss = 0.68158052
Iteration 4, loss = 0.66672913
Iteration 5, loss = 0.64261986
Iteration 6, loss = 0.59763393
Iteration 7, loss = 0.50344354
Iteration 8, loss = 0.34982035
Iteration 9, loss = 0.18737424
Iteration 10, loss = 0.09429573
Iteration 11, loss = 0.06038294
Iteration 12, loss = 0.04866964
Iteration 13, loss = 0.04410997
Iteration 14, loss = 0.04180204
Iteration 15, loss = 0.04022787
Iteration 16, loss = 0.03893421
Iteration 17, loss = 0.03733961
Iteration 18, loss = 0.03657173
Iteration 19, loss = 0.03535737
Iteration 20, loss = 0.03205988
Iteration 21, loss = 0.03007697
Iteration 22, loss = 0.03237719
Iteration 23, loss = 0.03248144
Iteration 24, loss = 0.03481693
Iteration 25, loss = 0.02949732
Iteration 26, loss = 0.02617889
Iteration 27, loss = 0.02856872
Iteration 28, loss = 0.02560701
Iteration 29, loss = 0.02881651
Iteration 30, loss = 0.02308987
Iteration 31, loss = 0.02350713
Iteration 32, los

In [3]:
# fft classification with sliding windows

from split_utils import split_text

all_docs_paras = [split_text(doc, segment_length=40) for doc in all_docs]

# remove any empty paragraphs
all_docs_paras = [[para for para in paras if para] for paras in all_docs_paras]
# remove any '' paragraphs
all_docs_paras = [[para for para in paras if para != ''] for paras in all_docs_paras]

In [20]:
idx = 58
print(all_docs_paras[idx][0])
print(all_labels[idx])

# print unique labels
print(set(train_labels))

SEOUL ANNOUNCES MORE TARIFF CUTS FOR U.S. South Korea will cut import taxes on 50 goods, including construction equipment, cigarettes and tobacco, to help reduce its trade surplus with the United States, the Finance Ministry said today. The tariff cuts
trade
{'dlr', 'hog', 'interest', 'coffee', 'coconut-oil', 'acq', 'cocoa', 'sugar', 'money-fx', 'ipi', 'cpi', 'cotton', 'iron-steel', 'gnp', 'bop', 'soybean', 'oilseed', 'jobs', 'castor-oil', 'trade', 'crude', 'carcass', 'grain', 'corn', 'reserves', 'lumber'}


In [4]:
# get embeddings for each paragraph
print('Getting embeddings...')
embeddings_engine = "text-embedding-ada-002"
train_embeddings_paras = [get_embeddings(paras, engine=embeddings_engine) for paras in tqdm(all_docs_paras)]

# pickle embeddings
with open('embeddings/all_embeddings_paras.pkl', 'wb') as f:
    pickle.dump(train_embeddings_paras, f)

Getting embeddings...


100%|██████████| 1063/1063 [02:51<00:00,  6.19it/s]


In [ ]:
# load embeddings
with open('embeddings/all_embeddings_paras.pkl', 'rb') as f:
    all_embeddings_paras = pickle.load(f)

# convert to numpy arrays
all_embeddings_paras = [np.array(doc) for doc in all_embeddings_paras]

# get FFTs
def get_fft(embedding):
    return np.abs(librosa.stft(embedding, n_fft=32, win_length=4))

# lowpass filter
def lowpass_filter(fft, cutoff=0.5):
    """
    Lowpass filter for FFTs
    """
    fft = fft.copy()
    fft[:, int(cutoff*fft.shape[1]):] = 0
    return fft

# convert back to embeddings
def fft_to_embedding(fft):
    return librosa.istft(fft, win_length=4)

# get FFTs
all_embeddings_paras_fft = [get_fft(embedding) for embedding in tqdm(all_embeddings_paras)]

# lowpass filter
# all_embeddings_paras_fft = [lowpass_filter(fft) for fft in tqdm(all_embeddings_paras_fft)]

# convert back to embeddings
all_embeddings_paras_lowpass = [fft_to_embedding(fft) for fft in tqdm(all_embeddings_paras_fft)]

# average embeddings
train_embeddings_lowpass_avg = [np.mean(embeddings, axis=0) for embeddings in all_embeddings_paras_lowpass]


In [4]:
# train/test split

X_train, X_test, y_train, y_test = train_test_split(train_embeddings_lowpass_avg, all_labels, test_size=0.2, random_state=42)

# train classifier
clf2 = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)
clf2.fit(X_train, y_train)

# predict on test set
y_pred = clf2.predict(X_test)

# evaluate
print(accuracy_score(y_test, y_pred))

Iteration 1, loss = 0.70230041
Iteration 2, loss = 0.68923404
Iteration 3, loss = 0.69466483
Iteration 4, loss = 0.69220395
Iteration 5, loss = 0.69040499
Iteration 6, loss = 0.69183168
Iteration 7, loss = 0.69260460
Iteration 8, loss = 0.69073242
Iteration 9, loss = 0.69174141
Iteration 10, loss = 0.69070615
Iteration 11, loss = 0.69060507
Iteration 12, loss = 0.69034749
Iteration 13, loss = 0.69082857
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
0.5633802816901409
